# Preperation

this section takes data and prepares it for CPR

## -BaseExtra-

this module includes basic items used in preperation

In [5]:
import random
import numpy as np
import pandas as pd

In [18]:
def testcheck(x, vset):
    if type(pd.DataFrame()) == pd.core.frame.DataFrame
        for v in vset.values:
            if np.equal(x, v).all().all():
                return True
    return False

In [ ]:
def closestvector(target, A):
    newv = [np.linalg.norm(np.array(np.array(target)-np.array(a))) for a in A]
    return np.argmin(newv)

In [2]:
def farthestvector(target, A):
    newv = [np.linalg.norm(np.array(np.array(target)-np.array(a))) for a in A]
    return np.argmax(newv)

In [ ]:
def move(x, a, v):
    n = abs(x-a)*v
    if a>x:
        return x+n
    elif a<x:
        return x-n
    else:
        return x

In [ ]:
def multimove(X, A, v):
    newvec = list()
    for n, x in enumerate(X):
        newvec.append(move(x, A[n], v))
    return newvec

## -DataPrepBase-

this is for preparing data

In [ ]:
def crush(vector):
    return [(v)/max(vector) for v in vector]

In [ ]:
def reduce(vector):
    return (np.array(vector)/np.sum(vector)).tolist()

## -Prep-

this is for preperation and as a branch to the other prep modules

In [ ]:
def scale(dataset):
    min_max_scaler = preprocessing.MinMaxScaler()
    return pd.DataFrame(min_max_scaler.fit_transform(dataset.values))

# Excecution

this section takes the data and trains the cpr model

## -Filter-

This module filters the dataset given decisions

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
def hardFilterFunction(cluster_function, pick, data_set):
    check = cluster_function.predict([pick])
    return data_set[cluster_function.predict(data_set)==check]

In [ ]:
def nearestFilterFunction(pick, data_set, cut = .50):
    cutt = int(round(len(data_set)*cut, 0))
    neigh = NearestNeighbors(n_neighbors=cutt)
    neigh.fit(data_set)
    return data_set.iloc[neigh.kneighbors([pick], cutt, return_distance=False)[0].tolist()]

## -Decision-

this module takes decisions and uses it to train a prediction module

In [10]:
import random

In [ ]:
def SimpleAveragePick(sets, cycles, target = 0):
    picklist = list()
    for c in range(cycles):
        frm = [sets[n].iloc[a] for n, a in enumerate([random.randint(1,len(s)-1) for s in sets])]
        for n, f in enumerate(frm):
            print('---Choice '+str(n)+'---')
            print(f)
        pick = frm[int(input('Pick From Choices: '))]
        picklist.append(pick)
    return np.average(picklist, axis=0)

## -SimBot-

this module simulates decisions based on a predetermined expected result

In [11]:
def SimpleAveragePick(target, sets, cycles):
    picklist = list()
    for c in range(cycles):
        frm = [sets[n].iloc[a] for n, a in enumerate([random.randint(1,len(s)-1) for s in sets])]
        pick = frm[BaseExtra.closestvector(target, frm)]
        picklist.append(pick)
    return np.average(picklist, axis=0)

In [12]:
def TriangleMove(target, sets, cycles):
    picklist = list()
    start = np.average([np.average(s, axis=0) for s in sets], axis=0)
    for c in range(cycles):
        frm = [sets[n].iloc[a] for n, a in enumerate([random.randint(1,len(s)-1) for s in sets])]
        close = frm[BaseExtra.closestvector(target, frm)]
        far = frm[BaseExtra.farthestvector(target, frm)]
        b = np.linalg.norm(np.array(far-close))/(np.linalg.norm(np.array(start-close))+np.linalg.norm(np.array(start-far)))
        start = np.array(BaseExtra.multimove(start, close, b))
    return start

## -Run-

this module runs Cluster Choice Filtering

In [18]:
def dataFilter(filter_function, pick, data_set, cluster_function = 0, cut = 0.50):
    if filter_function == None:
        raise Exception('Pick a data filter! available filters: hard, nearest')
    elif filter_function == 'hard':
        return Filter.hardFilterFunction(cluster_function, pick, data_set)
    elif filter_function == 'nearest':
        return Filter.nearestFilterFunction(pick, data_set, cut)

In [ ]:
def decisionFunction(decision_function):
    if decision_function == None:
        raise Exception('Pick a decision function!')

In [ ]:
def clusterChoiceFiltering(data_set,
                           cycles_per_epoch=10,
                           max_epochs=10,
                           decision_function='simpleAveragePick',
                           cluster_function='kMeans',
                           filter_function='nearset',
                           n_clusters=3,
                           minimum_cluster_size=10,
                           maximum_cluster_size=40,
                           target_array=None
                          ):
    decision_function = decisionFunction

In [ ]:
def clusterChoiceFiltering(data_set,
                           decision_function, 
                           cluster_function, 
                           filter_function='nearest', 
                           max_epochs, 
                           cycles_per_epoch = 0, 
                           target_array = 0, 
                           n_clusters=2, 
                           minimum_cluster_size=10, 
                           maximum_cluster_size=40
                          ):
    curset = data_set
    cfunc = cluster_function(n_clusters = n_clusters)
    cfunc.fit(curset)
    echeck = 0
    for e in range(max_epochs):
        if len(curset)<maximum_cluster_size:
            return curset, echeck
        else:
            if len(curset)>= n_clusters:
                if min([len(curset[cfunc.predict(curset)==n]) for n in range(0, n_clusters)])<3:
                    pass
                else:
                    if len(curset)<maximum_cluster_size:
                        pass
                    else:
                        echeck+=1
                        o = decision_function(target = target_array, sets = [curset[cfunc.predict(curset)==n] for n in range(0, n_clusters)], cycles = cycles_per_epoch)
                        curset = dataFilter(filter_function = filter_function, cluster_function=cfunc, pick = o, data_set = curset)
                        lastcurset = curset
                        try:
                            cfunc.fit(curset)
                        except:
                            return curset, echeck
                        if len(curset)<minimum_cluster_size:
                            return lastcurset, echeck
    return curset, echeck

## -DeepRun-

this module goes through all possible results with simbot

In [ ]:
import progressbar

In [ ]:
def clusterChoiceFiltering(decision_functions, 
    data_set, 
    cluster_function_list, 
    filter_function_list, 
    max_epochs, 
    cycles_per_epoch_list, 
    n_clusters_list, 
    minimum_cluster_size=10, 
    maximum_cluster_size=40,
    showprogress = True):
    check = list()
    step = 0
    with progressbar.ProgressBar(max_value=(len(data_set)*len(decision_functions)*len(cluster_function_list)*len(filter_function_list)*len(n_clusters_list)*len(cycles_per_epoch_list))) as bar:
        for dfun in decision_functions:
            for cfun in cluster_function_list:
                for ffun in filter_function_list:
                    for nclu in n_clusters_list:
                        for ccyc in cycles_per_epoch_list:
                            print([dfun, cfun, ffun, nclu, ccyc])
                            for g in range(0,len(data_set)):
                                if showprogress==True:
                                    step+=1
                                    bar.update(step)
                                clust = Run.clusterChoiceFiltering(decision_function = dfun,
                                    data_set = data_set, 
                                    target_array = data_set[g:g+1], 
                                    cluster_function = cfun, 
                                    filter_function = ffun, 
                                    max_epochs = max_epochs, 
                                    cycles_per_epoch = ccyc, 
                                    n_clusters = nclu, 
                                    minimum_cluster_size = minimum_cluster_size,
                                    maximum_cluster_size = maximum_cluster_size)
                                check.append([str(dfun), str(cfun), str(ffun), str(nclu), str(ccyc), data_set[g:g+1], clust[1], clust[0]])
    return pd.DataFrame(check, columns=['Decision_Function', 'Cluster_Function', 'Filter', 'N_Clusters', 'Cycles', 'Item', 'Epochs', 'Cluster'])

# Diagnostics

this section takes the cpr model and runs diagnosis on it

## -Graph-

this module makes a 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def alternatespace(x, y):
    group = list()
    for a in range(y):
        for b in range(x):
            group.append([b, a])
    return group

In [8]:
def multiclustgraph(dataset, size=50):
    a = len(dataset.columns)
    fig, axs = plt.subplots(a, a, figsize=(size, size))
    for c, t in enumerate(alternatespace(a, a)):
        if t[0] == t[1]:
            axs[t[0]][t[1]].hist(dataset[dataset.columns[[t[0]]]], color ='tab:blue', histtype = 'step', bins=100)
            axs[t[0]][t[1]].title.set_text(dataset.columns[t[0]]+' vs. '+dataset.columns[t[1]])
        else:
            if t[0]>t[1]:
                axs[t[0]][t[1]].scatter(dataset[dataset.columns[[t[0]]]],dataset[dataset.columns[t[1]]], color ='tab:blue')
                axs[t[0]][t[1]].title.set_text(dataset.columns[t[0]]+' vs. '+dataset.columns[t[1]])
                axs[t[0]][t[1]].set_xlabel(dataset.columns[t[0]])
                axs[t[0]][t[1]].set_ylabel(dataset.columns[t[1]])
            if t[0]<t[1]:
                axs[t[0]][t[1]].scatter(dataset[dataset.columns[[t[0]]]],dataset[dataset.columns[t[1]]], color ='tab:orange')
                axs[t[0]][t[1]].title.set_text(dataset.columns[t[0]]+' vs. '+dataset.columns[t[1]])
                axs[t[0]][t[1]].set_xlabel(dataset.columns[t[0]])
                axs[t[0]][t[1]].set_ylabel(dataset.columns[t[1]])
    fig.show()


[3, 3, 1, 3, 2]

In [ ]:
def diagnosePlot(frame, columns):
    for c in columns:
        m = list(range(1, max(frame[c])+2))
        plt.figure(figsize=(8, 6))
        plt.title('Distribution of Total '+c, fontsize=20)
        plt.xticks(m)
        plt.xlabel('Total '+c)
        plt.hist(frame[frame['Sucessful']==True][c], bins = m, align = 'left', stacked=True, label='Sucessful Test', color = 'blue')
        plt.hist(frame[frame['Sucessful']==False][c], bins = m, align = 'left', stacked=True, label='Un-Sucessful Test', color = 'orange')
        plt.legend()
        plt.text(0, -20, 'Minimum '+c+': '+str(np.min(frame[c]))+' | Average '+c+': '+str(round(np.average(frame[c]), 1))+' | Maximum '+c+': '+str(np.max(frame[c])), fontsize=15)
        plt.text(0, -40, 'Average '+c+': '+str(round(np.average(frame[c]), 1)), fontsize=15)
        plt.text(0, -60, 'Maximum '+c+': '+str(np.max(frame[c])), fontsize=15)
        plt.show()

## -Diagnose-

this checks the results of a deep run

In [ ]:
def deepVitals(DeepFull):
    check = list()
    for d in range(len(DeepFull)):
        check.append([BaseExtra.testcheck(DeepFull[d:d+1]['Item'].iloc[0], DeepFull[d:d+1]['Cluster'].iloc[0]), DeepFull[d:d+1]['Epochs'].iloc[0], len(DeepFull[d:d+1]['Cluster'].iloc[0])])
    checkframe = pd.DataFrame(check, columns=['Sucessful', 'Epochs', 'Cluster_Size'])
    Graph.diagnosePlot(checkframe, ['Epochs', 'Cluster_Size'])
    print(str(len(checkframe[checkframe['Sucessful']==True]))+' Sucessful out of '+str(len(checkframe))+" ("+str(round(len(checkframe[checkframe['Sucessful']==True])/len(checkframe), 3))+")")
    return checkframe